In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

print(X_train.isna().sum())
print(X_test.isna().sum())

X_train = X_train.drop('환불금액', axis = 1)
X_test = X_test.drop('환불금액', axis = 1)
# 사용자 코딩
X_train_cat = X_train.select_dtypes('object').copy()
X_test_cat =  X_test.select_dtypes('object').copy()

ohe = OneHotEncoder(sparse=False)

ohe.fit(X_train_cat)

X_train_ohe = ohe.transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

X_train_num = X_train.select_dtypes(exclude='object').copy()
X_test_num = X_test.select_dtypes(exclude='object').copy()

scaler = MinMaxScaler()

scaler.fit(X_train_num)

X_train_sca = scaler.transform(X_train_num)
X_test_sca = scaler.transform(X_test_num)

X_TRAIN = np.concatenate([X_train_ohe, X_train_sca], axis=1)
X_TEST = np.concatenate([X_test_ohe, X_test_sca], axis=1)

y_TRAIN = y_train['gender']

print(type(X_TRAIN), type(X_TEST), type(y_TRAIN))
print(X_TRAIN.shape, X_TEST.shape, y_TRAIN.shape)

xtrain, xtest, ytrain, ytest = train_test_split(X_TRAIN, y_TRAIN, test_size = 0.25, stratify=y_TRAIN, random_state=1234)

print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

def make_models(xtrain, xtest, ytrain, ytest):
    model1 = LogisticRegression().fit(xtrain, ytrain)
    print('model1', get_scores(model1, xtrain, xtest, ytrain, ytest))

    model2 = DecisionTreeClassifier(random_state=0).fit(xtrain, ytrain)
    print('model2', get_scores(model2, xtrain, xtest, ytrain, ytest))

    for d in range(3, 8):
        model2 = DecisionTreeClassifier(max_depth=d, random_state=0).fit(xtrain, ytrain)
        print('model2', d, get_scores(model2, xtrain, xtest, ytrain, ytest))

    model3 = RandomForestClassifier(random_state=0).fit(xtrain, ytrain)
    print('model3', get_scores(model3, xtrain, xtest, ytrain, ytest))

    for d in range(3, 8):
        model3 = RandomForestClassifier(500, max_depth=d, random_state=0).fit(xtrain, ytrain)
        print('model3', d, get_scores(model3, xtrain, xtest, ytrain, ytest))


def get_scores(model, xtrain, xtest, ytrain, ytest):
    A = model.score(xtrain, ytrain)
    
    ypred = model.predict_proba(xtest)[:, 1]
    
    B = roc_auc_score(ytest, ypred)
    
    return f'{A:.4} {B:.4}'
	
	
make_models(xtrain, xtest, ytrain, ytest)

final_model = RandomForestClassifier(max_depth=4, random_state=0).fit(xtrain, ytrain)

print('final model', get_scores(final_model, xtrain, xtest, ytrain, ytest))
#
# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)
